In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Rajeeb\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


## NASA Mars News

In [3]:
# URL of NASA Mars News Site page to be scraped
url_nasa = "https://mars.nasa.gov/news/"

# Open a browser window of the url page. 
browser.visit(url_nasa)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
html=browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Get the first instance of the target 
results = soup.select_one("ul.item_list li.slide")

In [6]:
# Identify and return the Latest News
title = results.find("div", class_="content_title").text

# Identify and return corresponding Paragraph Text 
para_text = results.find("div", class_="article_teaser_body").text
        
# Print the latest News Title and Paragraph Text
print(f'news_title = "{title}"')
print(f'news_paragraph = "{para_text}"')


news_title = "NASA's Perseverance Drives on Mars' Terrain for First Time"
news_paragraph = "The first trek of the agency’s largest, most advanced rover yet on the Red Planet marks a major milestone before science operations get under way."


## JPL Mars Space Images - Featured Image

In [7]:
# URL of JPL Mars Space page to be scraped
url_spaceImg = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url_spaceImg)

In [8]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Get the instance for Featured Image 
featured = soup.find('img', class_='headerimage fade-in')

# Store Featured image url in a variable and print the full url
featured_image_url = featured['src']
print('featured_image_url = https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + featured_image_url)

featured_image_url = https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


## Mars Facts

In [9]:
# Define url of USGS Astrogeology site and open in a browser
url_spaceFacts = 'https://space-facts.com/mars/'
browser.visit(url_spaceFacts)

In [10]:
# Use Panda's `read_html` to parse the url and get all tables
tables = pd.read_html(url_spaceFacts)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [11]:
# Find table containing facts about the planet including Diameter, Mass, etc and print it. 
df_marsFacts = tables[2]
df_marsFacts.columns = ['Features', 'Mars']
df_marsFacts

,Features,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
# Save table directly to a html file
df_marsFacts.to_html('marsFactsTable.html')

In [13]:
# Directly run the file in a browser. 
!explorer marsFactsTable.html

## Mars Hemispheres

In [14]:
# Visit url of USGS Astrogeology site and open in browser
base_url = 'https://astrogeology.usgs.gov/'
url_marsHS = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_marsHS)

# Scrape page into Soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
# Identify and return the data path
all_hems = soup.find('div', class_ = 'collapsible results')
hems = soup.find_all('div', class_='item')

In [16]:
# Get the image url and titles from soup
hemisphere_image_urls = []
for hem in hems:
    try:
        hem_data = hem.find('div', class_='description')  
        title = hem_data.a.h3.text
        
        # Get full image link url and scrape data
        hemPic_url = hem_data.a['href']
        browser.visit(base_url+hemPic_url)
        
        # Scrape page into Soup
        html = browser.html
        img_soup = BeautifulSoup(html, 'html.parser')
        
        # Identify and return the data path
        img_src1=img_soup.find('div', class_='downloads')
        img_src = img_src1.find('li').a['href']
        
        # Print scrapped data only if both are found
        if (title and img_src):
            print('------------------------------------')
            print(title)
            print(img_src)
            
        # Populate data in a dictionary
        hem_dict={
            'title':title,
            'image_url':img_src
        }
        
        # Update list after each data retrieval
        hemisphere_image_urls.append(hem_dict)
    except Exception as e:
        print(e) 


------------------------------------
Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
------------------------------------
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
------------------------------------
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
------------------------------------
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [17]:
# Close the browser after scraping
browser.quit()